In [1]:
import json
import datetime

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from geopy.distance import distance
from tqdm import tqdm

plt.rc('font',family='AppleGothic')

In [2]:
df_raw = pd.read_csv("./data/main_data/main_df.csv")
df = df_raw.copy()

In [3]:
with open("./data/assets/address2coordinate.json", 'r') as f:
    address_coordinate_dict=json.load(f)

In [4]:
df_addr = df.loc[:,['주소']].drop_duplicates()

In [5]:
df_apt_coordinate = pd.concat([df_addr,df_addr['주소'].map(address_coordinate_dict).apply(pd.Series)],axis=1)
df_apt_coordinate.rename(columns={'주소':'name'},inplace=True)
df_apt_coordinate = df_apt_coordinate.loc[:,['name','lat','lng']]

df_apt_coordinate = df_apt_coordinate[
    (df_apt_coordinate["lat"] != 0) & (df_apt_coordinate["lng"] != 0)
]

In [6]:
df_hanriver_park_coordinate = pd.read_csv("./data/sub_data/hanriver.csv")
df_hanriver_park_coordinate.rename(columns={'공원명':'name','위도':'lat','경도':'lng'},inplace=True)

df_hanriver_park_coordinate = df_hanriver_park_coordinate.loc[:,['name','lat','lng']]

In [7]:
df_subway_coordinate = pd.read_csv("./data/sub_data/station_coordinate.csv")
df_subway_coordinate = df_subway_coordinate.loc[:,['name','lat','lng']]
df_subway_coordinate.dropna(inplace=True)

In [8]:
address2hanriver = dict()
address2subway = dict()
for name, lat, lng in tqdm(df_apt_coordinate.values):
    # hanriver distance
    distance2hanriver = df_hanriver_park_coordinate.apply(lambda x : distance((x.lat,x.lng),(lat,lng)).km,axis=1).min()
    address2hanriver[name] = distance2hanriver
    
    # subway distance
    distance2subway = df_subway_coordinate.apply(lambda x : distance((x.lat,x.lng),(lat,lng)).km,axis=1).min()
    address2subway[name] = distance2subway

100%|██████████| 8925/8925 [08:18<00:00, 17.92it/s]


In [10]:
with open("./data/assets/address2hanriver.json",'w') as f:
    json.dump(address2hanriver,f,indent=4,ensure_ascii=False)
with open("./data/assets/address2subway.json",'w') as f:
    json.dump(address2subway,f,indent=4,ensure_ascii=False)